In [1]:
require 'redis'
require 'date'
require 'pathname'

redis = Redis.new(host: "redis", port: 6379, db: 0)

#<Redis client v4.3.1 for redis://redis:6379/0>

In [2]:
#define start month (vts) and end month (vte) for PRR computation
END_MONTH=[2017,12]
START_MONTH=[2017,1] #1 year version
#START_MONTH=[2008,1] #10 years version


def vt_next_month(vt, length)
  years = (length / 12) + vt[0]
  r = [vt[0], nil]
  if vt[1] + (length % 12) > 12
    r[0] = years + 1
    r[1] = vt[1] + (length % 12) - 12
  else 
    r[0] = years 
    r[1] = vt[1] + (length % 12)
  end  
  r
end

:vt_next_month

In [5]:
# FUNCTION TO COMPUTE PRR

#vts and vte are included in interval: [vts,vte]
def prr_compute(pair, redis, vts, vte)
  a,b,c,d = 0,0,0,0
  (vts[0]..vte[0]).each do |year|
      s_month = vts[0] == year ? vts[1] : 1
      e_month = vte[0] == year ? vte[1] : 12
      (s_month..e_month).each do |month|
             ca = redis.get({
                  incremental: false, 
                  year: year,
                  month: month,
                  drug: pair[:drug],
                  soc: pair[:soc]
                 }.to_s)
              a += ca.nil? ? 0.0 : ca.to_f
        
              cb = redis.get({
                  incremental: false, 
                  year: year,
                  month: month,
                  drug: pair[:drug]
                 }.to_s)
              b += cb.nil? ? 0.0 : cb.to_f
        
              cc = redis.get({
                  incremental: false, 
                  year: year,
                  month: month,
                  soc: pair[:soc]
                 }.to_s)
              c += cc.nil? ? 0.0 : cc.to_f
        
              cd = redis.get({
                  incremental: false, 
                  year: year,
                  month: month
                 }.to_s)  
              d += cd.nil? ? 0.0 : cd.to_f
      end  
  end  
  s = Math.sqrt((1/a) + (1/c) + (-1/(a +b)) + (-1/(c+d)))
  prr = (a/c)/((b - a)/d)
  {
    a: a,
    b: b,
    c: c,
    d: d,
    s: s,
    prr: prr,
    count_cases: a,
    confidence: [prr/ Math.exp(1.96 * s), prr * Math.exp(1.96 * s)],
    chi_square: ((a*d - b*c)**2)*(a+b+c+d)/((a+b)*(c+d)*(a+c)*(b+d))
    }
end

:prr_compute

In [ ]:
## Example of prr_compute function for a single pair in interval [2017-06-01, 2017-12-31]

pairs = redis.keys({incremental: true, year: END_MONTH[0], month: END_MONTH[1], drug: "*", soc: "*"}.to_s); nil
pairs = pairs.map{|p| h = (eval p); {drug: h[:drug], soc: h[:soc]} }.uniq
pair= pairs.sample
puts pair

prr_compute(pair, redis, [2017,6], [2017,12])

In [ ]:
#TEDAR (MONTHLY AS TIME UNITS)

def start_month(month)
  monthToAdd=(month-1)%12
  year=(month-1)/12.to_i
  [START_MONTH[0]+year, START_MONTH[1]+monthToAdd]
end

def end_month(month)
  start_month(month-1)
end
  

Pathname('results/TEDAR/result_TEDAR.txt').dirname.mkpath


File.readlines('results/TEDAR/split/split_TEDAR.txt').each do |line|
  array=line.strip.split("\t")
  pair = { :drug => array[0], :soc => array[1] } 
  sp=array[2].split(",").map(&:to_i)
  sp.each_with_index{ |x,index| 
    if index != sp.size-1 
      if (x == 0 && sp[index+1] !=1)
        x=1
      end
      if (x != 0)
      res=prr_compute(pair,redis,start_month(x),end_month(sp[index+1]))
      #drug,soc,vts,vte,prr, confidence-,confidence+,chi, count
      File.write("results/TEDAR/result_TEDAR.txt", pair[:drug]+"\t"+pair[:soc]+"\t"+start_month(x).to_s+"\t"+end_month(sp[index+1]).to_s+"\t"+res[:prr].to_s+"\t"+res[:confidence][0].to_s+"\t"+res[:confidence][1].to_s+"\t"+res[:chi_square].to_s+"\t"+res[:count_cases].to_i.to_s+"\n", mode: "a")
      end
    end
    }
end

In [6]:
# PRR MONTHLY

#vts and vte are included: [vts,vte]
def prr_monthly(pair, redis, vts, vte)
  current = vts.clone()
  r=[]
  while current != vte
    r.push( prr_compute(pair, redis, current, current).merge({month: [current[0],current[1]]}))
    if current[1] == 12 
      current[1]  = 1
      current[0] += 1
    else  
      current[1] += 1
    end  
  end  
  r.push(prr_compute(pair, redis, current, current).merge({ month: [current[0],current[1]]})) 
end 


Pathname('results/PRRmonthly/result_prr_monthly.txt').dirname.mkpath


redis.keys({incremental: true,  year: END_MONTH[0], month: END_MONTH[1], drug: "*", soc: "*"}.to_s).each do |key_string|
   pair = eval key_string
   res=prr_monthly(pair,redis,START_MONTH,END_MONTH)
   res.each{|val| 
     File.write("results/PRRmonthly/result_prr_monthly.txt", pair[:drug].to_s+"\t"+pair[:soc].to_s+"\t"+val[:month].to_s+"\t"+val[:month].to_s+"\t"+val[:prr].to_s+"\t"+val[:confidence][0].to_s+"\t"+val[:confidence][1].to_s+"\t"+val[:chi_square].to_s+"\t"+val[:count_cases].to_i.to_s+"\n", mode: "a")
     }
end

Interrupt: 

In [7]:
#PRR QUARTERLY

#vts and vte are included: [vts,vte]
def prr_quarterly(pair, redis, vts, vte)
  current = vts.clone()
  next_current = Array.new(2,0)
  next_current[0]=current[0]
  next_current[1]=current[1]+2
  r=[]
  while next_current != vte
    r.push( prr_compute(pair, redis, current,next_current).merge({vts: [current[0],current[1]]}).merge({vte: [next_current[0],next_current[1]]}))
    if current[1] == 10 
      current[1]  = 1
      current[0] += 1
    else  
      current[1] += 3
    end
    next_current[0]=current[0]
    next_current[1]=current[1]+2
  end  
    r.push( prr_compute(pair, redis, current,next_current).merge({vts: [current[0],current[1]]}).merge({vte: [next_current[0],next_current[1]]}))
end 


Pathname('results/PRRquarterly/result_prr_quarterly.txt').dirname.mkpath


redis.keys({incremental: true, year: END_MONTH[0], month: END_MONTH[1], drug: "*", soc: "*"}.to_s).each do |key_string|
   pair = eval key_string
   res=prr_quarterly(pair,redis, START_MONTH,END_MONTH)
   res.each{|val| 
     File.write("results/PRRquarterly/result_prr_quarterly.txt", pair[:drug].to_s+"\t"+pair[:soc].to_s+"\t"+val[:vts].to_s+"\t"+val[:vte].to_s+"\t"+val[:prr].to_s+"\t"+val[:confidence][0].to_s+"\t"+val[:confidence][1].to_s+"\t"+val[:chi_square].to_s+"\t"+val[:count_cases].to_i.to_s+"\n", mode: "a")
    }
end

Interrupt: 

In [9]:
# PRR YEARLY 

#vts included, vte exlcuded: [vts, vte[
def prr_yearly(pair, redis, vts, vte)
  current = vts.clone()
  next_current = Array.new(2,0)
  r = []
  while current[0] < vte[0]
    if (current[1] == 1)
      next_current[1] = 12
      next_current[0] = current[0]
    else
      next_current[1] = current[1] - 1
      next_current[0] = current[0] + 1
    end      
    r.push( prr_compute(pair, redis, current, next_current).merge({vts: [current[0],current[1]]}).merge({vte: [next_current[0],next_current[1]]}))
    current[0] += 1
  end  
  r
end

Pathname('results/PRRyearly/result_prr_yearly.txt').dirname.mkpath


redis.keys({incremental: true, year: END_MONTH[0], month: END_MONTH[1], drug: "*", soc: "*"}.to_s).each do |key_string|
   pair = eval key_string
   res=prr_yearly(pair,redis,START_MONTH, vt_next_month(END_MONTH,1))
   res.each{|val| 
     File.write("results/PRRyearly/result_prr_yearly.txt", pair[:drug].to_s+"\t"+pair[:soc].to_s+"\t"+val[:vts].to_s+"\t"+val[:vte].to_s+"\t"+val[:prr].to_s+"\t"+val[:confidence][0].to_s+"\t"+val[:confidence][1].to_s+"\t"+val[:chi_square].to_s+"\t"+val[:count_cases].to_i.to_s+"\n", mode: "a")
     }
end

Interrupt: 